In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [2]:
train_data = pd.read_csv("./smile_description_train.csv")
test_data = pd.read_csv("./smile_description_test.csv")

In [3]:
# Replacing Null values of train and test data with 0 
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [4]:
# Split into X and y
X = train_data.drop("label",axis=1)
y = train_data["label"]

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [6]:
def create_submission(predict,filename):
    sub_file = pd.read_csv("./data/sample_submission.csv")
    sub_file["Predicted"] = predict
    sub_file.to_csv(filename,index=False)
    print(filename," Created")

In [20]:
ann = MLPClassifier(random_state=42,hidden_layer_sizes=20,verbose=True,max_iter=20)

In [9]:
def f1_score_xg_boost(X,y,learning_rate=None):
    skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    cv =  skf.get_n_splits(X, y)
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
    cv_results = cross_validate(xgb.XGBClassifier(random_state=42,n_estimators=700,eta=learning_rate), X, y, cv=cv,scoring=scoring,verbose=3,n_jobs=-1)
    print("F1 score with ",sum(cv_results["test_f1_score"])/5)
    print("Accuracy score with ",sum(cv_results["test_accuracy"])/5)

In [10]:
import numpy as np
def get_count_of_ones_and_twos(predict):
    print("Number of predicted ones",np.count_nonzero(predict==1))
    print("Number of predicted twos",np.count_nonzero(predict==2))

In [22]:
# f1_score_xg_boost(X,y,0.35)
"""
n_estimators = 700
learning_rate = 0.35
F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841
"""

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 11.5min remaining: 17.3min


F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 11.6min finished


In [23]:
# xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=700,eta=0.35)

[CV] END  accuracy: (test=0.914) f1_score: (test=0.950) precision: (test=0.936) recall: (test=0.965) total time=35.3min
[CV] END  accuracy: (test=0.911) f1_score: (test=0.949) precision: (test=0.938) recall: (test=0.960) total time=11.5min
[CV] END  accuracy: (test=0.909) f1_score: (test=0.948) precision: (test=0.934) recall: (test=0.961) total time=35.3min
[CV] END  accuracy: (test=0.908) f1_score: (test=0.947) precision: (test=0.935) recall: (test=0.960) total time=11.5min
[CV] END  accuracy: (test=0.909) f1_score: (test=0.948) precision: (test=0.935) recall: (test=0.962) total time=35.3min
[CV] END  accuracy: (test=0.907) f1_score: (test=0.946) precision: (test=0.934) recall: (test=0.959) total time=11.5min
[CV] END  accuracy: (test=0.915) f1_score: (test=0.951) precision: (test=0.938) recall: (test=0.964) total time=35.3min
[CV] END  accuracy: (test=0.913) f1_score: (test=0.950) precision: (test=0.936) recall: (test=0.964) total time=11.5min
[CV] END  accuracy: (test=0.905) f1_scor

In [21]:
ann.fit(X,y)
predict = ann.predict(test_data)
predict_real = label_encoder.inverse_transform(predict)

Iteration 1, loss = 8.52129639
Iteration 2, loss = 7.12214624
Iteration 3, loss = 7.87804814
Iteration 4, loss = 7.59917941
Iteration 5, loss = 7.54881294
Iteration 6, loss = 7.40288529
Iteration 7, loss = 7.89212516
Iteration 8, loss = 6.84181302
Iteration 9, loss = 7.78191080
Iteration 10, loss = 7.59097725
Iteration 11, loss = 7.50959502
Iteration 12, loss = 7.67771050
Iteration 13, loss = 7.04938328
Iteration 14, loss = 7.84240255
Iteration 15, loss = 7.39539437
Iteration 16, loss = 8.15235200
Iteration 17, loss = 6.89217228
Iteration 18, loss = 6.69757648
Iteration 19, loss = 7.25708492
Iteration 20, loss = 6.97957636


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
get_count_of_ones_and_twos(predict_real)

Number of predicted ones 1667
Number of predicted twos 9327


In [23]:
create_submission(predict_real,"submission_1_mar_1.csv")

submission_1_mar_1.csv  Created
